<a href="https://colab.research.google.com/github/asabbah44/Text_Classification/blob/main/Word_Emb_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.nbshare.io/notebook/197284676/Word-Embeddings-Transformers-In-SVM-Classifier-Using-Python/

In [ ]:
!pip install tensorflow_text
import tensorflow_text

In [ ]:
import numpy as np
import tensorflow_hub as hub
import tensorflow_text  # this needs to be imported to set up some stuff in the background

In [ ]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

In [ ]:
import re

import pandas as pd

In [ ]:
import string


from spacy.lang.en import stop_words as spacy_stopwords  # we use spacy's list of stop words to clean our data


stop_words = spacy_stopwords.STOP_WORDS
punctuations = string.punctuation



def clean(text):
    text = ' '.join(word for word in text.split() if word not in stop_words)
    text = re.sub(r'\W+', ' ', text)  # remove non-alphanumeric characters
    # replace numbers with the word 'number'
    text = re.sub(r"\d+", "number", text)
    # don't consider sentenced with less than 3 words (i.e. assumed noise)
    if len(text.strip().split()) < 1:
        return None
    text = text.lower()  # lower case everything
    
    return text.strip() # remove redundant spaces

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/dataset.csv')
df2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/mydatasetfinal.csv')


df1 = pd.DataFrame(df1, columns = ['commenttext','label'])
df2 = pd.DataFrame(df2, columns = ['commenttext','label'])

df=df1#pd.concat([df1,df2])

#df = df.assign(clean_text=df.commenttext.apply(clean)).dropna()

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks')

from Utility import preprocess
df["commenttext"] = df["commenttext"].apply(lambda x: x.lower())

# # Clean the text
df["commenttext"] = df["commenttext"].apply(lambda x:preprocess.clean_text(x))

# # Clean numbers
df["commenttext"] = df["commenttext"].apply(lambda x:preprocess.clean_numbers(x))

# # Clean Contractions
df["commenttext"] = df["commenttext"].apply(lambda x:preprocess.replace_contractions(x))
 
# df["commenttext"] = df["commenttext"].apply(lambda x:preprocess.clean_punc(x))

# # lemmatize
df["commenttext"] = df["commenttext"].apply(lambda x: preprocess.lemmatize(x))
# # remove stop word and custome word such as todo
# df["commenttext"] = df["commenttext"].apply(lambda x: preprocess.custm_stopwords_remove(x))

In [ ]:
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
nltk.download('stopwords')
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text

    Cust_words = []
    #Cust_words = []
    stop = set(stopwords.words('english'))
    for word in  Cust_words:
        STOPWORDS.add(word)
    text = ' '.join(word for word in text.split() if word not in stop)

    return text
    
df['commenttext'] = df['commenttext'].apply(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [ ]:
# we split the data into train and test
X = df.commenttext
y = df.label
msg_train, msg_test, y_train, y_test = train_test_split(X, y,test_size=0.2,random_state=42)

In [ ]:
# we just feed in the list of sentences, and we get the vector representation of each sentence
X_test = embed(msg_test)
X_test.shape

print(X_test)

tf.Tensor(
[[-0.0648374  -0.03300815  0.05215542 ...  0.06500135  0.01990111
   0.02063693]
 [ 0.09466336  0.03706772  0.01326222 ...  0.02647947 -0.01125136
   0.02540344]
 [-0.07777959  0.04419092  0.02880645 ...  0.01593046 -0.03701168
   0.05171212]
 ...
 [ 0.07155395  0.03866101 -0.02804634 ... -0.07812867 -0.01942797
   0.00308762]
 [ 0.06828818 -0.00321373  0.04473334 ... -0.02397262 -0.03765468
  -0.00602519]
 [ 0.09337119  0.0592142  -0.01672832 ... -0.01004277  0.03576313
   0.09544707]], shape=(815, 512), dtype=float32)


In [ ]:
# we don't have enough memory to apply embeddings in one shot,
# so we have to split the data into batches and concatenate them later
splits = np.array_split(msg_train, 5)
l = list()
for split in splits:
    l.append(embed(split))

In [ ]:
X_train = tf.concat(l, axis=0)
del l
X_train.shape

TensorShape([3256, 512])

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
class_weight = compute_class_weight(
    class_weight='balanced', classes=["Requirement","Design","Defect","Test","Documentation"], y=y_train
)
class_weight


array([ 1.07458746,  0.30401494,  1.69142857,  8.68266667, 13.56666667])

In [ ]:
class_weight={"Requirement":class_weight[0], "Design":class_weight[1],"Defect":class_weight[2],"Test":class_weight[3],
                        "Documentation":class_weight[4]}

In [ ]:
from sklearn.svm import SVC,LinearSVC,LinearSVR
import time
for i in range(0,10):
 linearSVC =  LinearSVC()
               
 linearSVC.fit(X_train, y_train)
 t0=time.clock()
 y_pred = linearSVC.predict(X_test)
 print("--------------------------",i)
 print(" secand %.3f" % float(time.clock()-t0))
 print('accuracy %.3f' % float(accuracy_score(y_pred, y_test)))
 print(classification_report(y_test, y_pred,digits=3))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  # Remove the CWD from sys.path while we load stuff.


-------------------------- 0
 secand 0.009
accuracy 0.753
               precision    recall  f1-score   support

       Defect      0.596     0.322     0.418        87
       Design      0.776     0.927     0.845       561
Documentation      0.750     0.500     0.600         6
  Requirement      0.690     0.384     0.494       151
         Test      0.500     0.500     0.500        10

     accuracy                          0.753       815
    macro avg      0.662     0.527     0.571       815
 weighted avg      0.737     0.753     0.728       815



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  # Remove the CWD from sys.path while we load stuff.


-------------------------- 1
 secand 0.007
accuracy 0.753
               precision    recall  f1-score   support

       Defect      0.596     0.322     0.418        87
       Design      0.776     0.927     0.845       561
Documentation      0.750     0.500     0.600         6
  Requirement      0.690     0.384     0.494       151
         Test      0.500     0.500     0.500        10

     accuracy                          0.753       815
    macro avg      0.662     0.527     0.571       815
 weighted avg      0.737     0.753     0.728       815



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  # Remove the CWD from sys.path while we load stuff.


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  # Remove the CWD from sys.path while we load stuff.


-------------------------- 7
 secand 0.011
accuracy 0.753
               precision    recall  f1-score   support

       Defect      0.596     0.322     0.418        87
       Design      0.776     0.927     0.845       561
Documentation      0.750     0.500     0.600         6
  Requirement      0.690     0.384     0.494       151
         Test      0.500     0.500     0.500        10

     accuracy                          0.753       815
    macro avg      0.662     0.527     0.571       815
 weighted avg      0.737     0.753     0.728       815



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  # Remove the CWD from sys.path while we load stuff.


-------------------------- 8
 secand 0.008
accuracy 0.753
               precision    recall  f1-score   support

       Defect      0.596     0.322     0.418        87
       Design      0.776     0.927     0.845       561
Documentation      0.750     0.500     0.600         6
  Requirement      0.690     0.384     0.494       151
         Test      0.500     0.500     0.500        10

     accuracy                          0.753       815
    macro avg      0.662     0.527     0.571       815
 weighted avg      0.737     0.753     0.728       815

-------------------------- 9
 secand 0.008
accuracy 0.753
               precision    recall  f1-score   support

       Defect      0.596     0.322     0.418        87
       Design      0.776     0.927     0.845       561
Documentation      0.750     0.500     0.600         6
  Requirement      0.690     0.384     0.494       151
         Test      0.500     0.500     0.500        10

     accuracy                          0.753       815

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
from sklearn.ensemble import RandomForestClassifier
for i in range(0,10):
 clf = RandomForestClassifier()
 clf.fit(X_train, y_train)
 t0=time.clock()
 y_pred = clf.predict(np.array(X_test))
 print(" secand %.3f" % float(time.clock()-t0))
 print(classification_report(y_test, y_pred,digits=3))
 print("------------------------",i)
 print('accuracy %s' % accuracy_score(y_pred, y_test))
 print('accuracy %s' % accuracy_score(y_pred, y_test))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  import sys


 secand 0.037
               precision    recall  f1-score   support

       Defect      0.933     0.322     0.479        87
       Design      0.783     0.993     0.876       561
Documentation      1.000     0.333     0.500         6
  Requirement      0.957     0.444     0.606       151
         Test      1.000     0.200     0.333        10

     accuracy                          0.805       815
    macro avg      0.935     0.458     0.559       815
 weighted avg      0.836     0.805     0.774       815

------------------------ 0
accuracy 0.8049079754601227
accuracy 0.8049079754601227


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  import sys


 secand 0.036
               precision    recall  f1-score   support

       Defect      0.933     0.322     0.479        87
       Design      0.785     0.993     0.876       561
Documentation      1.000     0.333     0.500         6
  Requirement      0.957     0.444     0.606       151
         Test      1.000     0.300     0.462        10

     accuracy                          0.806       815
    macro avg      0.935     0.478     0.585       815
 weighted avg      0.837     0.806     0.776       815

------------------------ 1
accuracy 0.8061349693251534
accuracy 0.8061349693251534


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  import sys


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  import sys


 secand 0.036
               precision    recall  f1-score   support

       Defect      0.933     0.322     0.479        87
       Design      0.782     0.991     0.874       561
Documentation      1.000     0.500     0.667         6
  Requirement      0.942     0.430     0.591       151
         Test      1.000     0.200     0.333        10

     accuracy                          0.802       815
    macro avg      0.931     0.489     0.589       815
 weighted avg      0.832     0.802     0.771       815

------------------------ 7
accuracy 0.8024539877300614
accuracy 0.8024539877300614


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  import sys


 secand 0.035
               precision    recall  f1-score   support

       Defect      0.933     0.322     0.479        87
       Design      0.783     0.993     0.876       561
Documentation      1.000     0.333     0.500         6
  Requirement      0.957     0.437     0.600       151
         Test      1.000     0.300     0.462        10

     accuracy                          0.805       815
    macro avg      0.935     0.477     0.583       815
 weighted avg      0.836     0.805     0.774       815

------------------------ 8
accuracy 0.8049079754601227
accuracy 0.8049079754601227
 secand 0.036
               precision    recall  f1-score   support

       Defect      0.933     0.322     0.479        87
       Design      0.782     0.993     0.875       561
Documentation      1.000     0.333     0.500         6
  Requirement      0.956     0.430     0.594       151
         Test      1.000     0.300     0.462        10

     accuracy                          0.804       815
    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  import sys


In [ ]:
from sklearn.naive_bayes import BernoulliNB
for i in range(0,10):
 clf = BernoulliNB()
 clf.fit(X_train, y_train)
 t0=time.clock()

 y_pred = clf.predict(np.array(X_test))
 print(" secand %.3f" % float(time.clock()-t0))
 print(classification_report(y_test, y_pred,digits=3))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


 secand 0.030
               precision    recall  f1-score   support

       Defect      0.466     0.310     0.372        87
       Design      0.780     0.720     0.749       561
Documentation      0.333     0.500     0.400         6
  Requirement      0.378     0.523     0.439       151
         Test      0.381     0.800     0.516        10

     accuracy                          0.639       815
    macro avg      0.468     0.571     0.495       815
 weighted avg      0.664     0.639     0.646       815



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


 secand 0.030
               precision    recall  f1-score   support

       Defect      0.466     0.310     0.372        87
       Design      0.780     0.720     0.749       561
Documentation      0.333     0.500     0.400         6
  Requirement      0.378     0.523     0.439       151
         Test      0.381     0.800     0.516        10

     accuracy                          0.639       815
    macro avg      0.468     0.571     0.495       815
 weighted avg      0.664     0.639     0.646       815



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


 secand 0.030
               precision    recall  f1-score   support

       Defect      0.466     0.310     0.372        87
       Design      0.780     0.720     0.749       561
Documentation      0.333     0.500     0.400         6
  Requirement      0.378     0.523     0.439       151
         Test      0.381     0.800     0.516        10

     accuracy                          0.639       815
    macro avg      0.468     0.571     0.495       815
 weighted avg      0.664     0.639     0.646       815



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


 secand 0.030
               precision    recall  f1-score   support

       Defect      0.466     0.310     0.372        87
       Design      0.780     0.720     0.749       561
Documentation      0.333     0.500     0.400         6
  Requirement      0.378     0.523     0.439       151
         Test      0.381     0.800     0.516        10

     accuracy                          0.639       815
    macro avg      0.468     0.571     0.495       815
 weighted avg      0.664     0.639     0.646       815

 secand 0.030
               precision    recall  f1-score   support

       Defect      0.466     0.310     0.372        87
       Design      0.780     0.720     0.749       561
Documentation      0.333     0.500     0.400         6
  Requirement      0.378     0.523     0.439       151
         Test      0.381     0.800     0.516        10

     accuracy                          0.639       815
    macro avg      0.468     0.571     0.495       815
 weighted avg      0.664     0.6

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


In [ ]:
from keras.layers import Input
from keras.models import Model
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import GlobalMaxPooling1D
from keras.optimizers import Adam
import keras
model = Sequential()
model.add(
  keras.layers.Dense(
    units=160,
    input_shape=(X_train.shape[1], ),
    activation='relu'
  )
)
model.add(
  keras.layers.Dropout(rate=0.2)
)
model.add(
  keras.layers.Dense(
    units=160,
    activation='relu'
  )
)
model.add(
  keras.layers.Dropout(rate=0.2)
)
model.add(keras.layers.Dense(5, activation='softmax'))


In [ ]:
# model.compile(loss='categorical_crossentropy', optimizer="Adam", metrics=['accuracy'])


In [ ]:
# from sklearn.preprocessing import OneHotEncoder

# type_one_hot = OneHotEncoder(sparse=False).fit_transform(
#   df.label.to_numpy().reshape(-1, 1))

In [ ]:
# train_x, test_x, train_y, test_y =\
#   train_test_split(
#     df.commenttext,
#     type_one_hot,
#     test_size=.2,
#     random_state=42
#   )

In [ ]:
# from tqdm import tqdm
# x_train= []
# for r in tqdm(train_x):
#   emb = embed(r)
#   x_emb = tf.reshape(emb, [-1]).numpy()
#   x_train.append(x_emb)

# x_train = np.array(x_train)

# x_test = []
# for r in tqdm(test_x):
#   emb = embed(r)
#   x_emb = tf.reshape(emb, [-1]).numpy()
#   x_test.append(x_emb)

# x_test = np.array(x_test)

In [ ]:
# history1 = model.fit(x_train, train_y, epochs=10, verbose=1, validation_split=0.2 )

# score = model.evaluate(x_test, test_y, verbose=1)